# Kaggle: Michael Jordan, Kobe Bryant and Lebron James stats

## Introducción

Para este proyecto se utilizaran diferentes bases de datos que proporcionan información sobre todas las estadisticas de tres de los mejores jugadores de la historia del baloncesto: Michael Jordan, Kobe Bryant y Lebron James.

Los datasets utilizados para este proyecto serán:

    1. allgames_stats.csv: BD que proporciona las estadísticas de todos los partidos de los 3 jugadores
    2. per_game_stats.csv: BD que proporciona las estadísticas medias de cada temporada de los 3 jugadores
    3. 
    

## Objetivos

Los objetivos de este proyecto serán dos. Por una banda, realizar un analisis de los datos del dataset donde se encuentran las estadisticas de cada jugador por temporada y de este modo extraer conclusiones. Por otra banda, realizar la predicción de un atributo tomandolo como el "target" del problema, es decir, realizar un problema de regresión logística o linear.

De esta forma el objetivo principal de este proyecto será contestar las siguientes 2 preguntas:
    
    1. ¿Quién es estadisticamente el mejor jugador de los tres? -> Analisis de datos
    2. ¿Cuál será el resultado de un partido? -> Predicción de atributo categórico (Regresión logística)
    
#preguntar que atributo es mejor predecir si regresion logistica (victoria o derrota) o regresion linear (salarios o puntos en un partido por ejemplo)

In [29]:
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import scipy.stats
import seaborn as sns

def load_dataset(path):
    dataset = pd.read_csv(path, header=0, delimiter=',')
    return dataset

# carreguem dataset 
dataset = load_dataset('per_game_stats.csv')
data = dataset.values

# @ -> visitante

In [30]:
dataset[dataset.Player == "Kobe Bryant"]

,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player,RSorPO
57,1996-97,18,LAL,NBA,SG,71,6,15.5,2.5,5.9,...,1.2,1.9,1.3,0.7,0.3,1.6,1.4,7.6,Kobe Bryant,Regular Season
58,1997-98,19,LAL,NBA,SG,79,1,26.0,4.9,11.6,...,2.1,3.1,2.5,0.9,0.5,2.0,2.3,15.4,Kobe Bryant,Regular Season
59,1998-99,20,LAL,NBA,SG,50,50,37.9,7.2,15.6,...,4.2,5.3,3.8,1.4,1.0,3.1,3.1,19.9,Kobe Bryant,Regular Season
60,1999-00,21,LAL,NBA,SG,66,62,38.2,8.4,17.9,...,4.7,6.3,4.9,1.6,0.9,2.8,3.3,22.5,Kobe Bryant,Regular Season
61,2000-01,22,LAL,NBA,SG,68,68,40.9,10.3,22.2,...,4.3,5.9,5.0,1.7,0.6,3.2,3.3,28.5,Kobe Bryant,Regular Season
62,2001-02,23,LAL,NBA,SG,80,80,38.3,9.4,20.0,...,4.1,5.5,5.5,1.5,0.4,2.8,2.9,25.2,Kobe Bryant,Regular Season
63,2002-03,24,LAL,NBA,SG,82,82,41.5,10.6,23.5,...,5.6,6.9,5.9,2.2,0.8,3.5,2.7,30.0,Kobe Bryant,Regular Season
64,2003-04,25,LAL,NBA,SG,65,64,37.6,7.9,18.1,...,3.9,5.5,5.1,1.7,0.4,2.6,2.7,24.0,Kobe Bryant,Regular Season
65,2004-05,26,LAL,NBA,SG,66,66,40.7,8.7,20.1,...,4.5,5.9,6.0,1.3,0.8,4.1,2.6,27.6,Kobe Bryant,Regular Season
66,2005-06,27,LAL,NBA,SG,80,80,41.0,12.2,27.2,...,4.4,5.3,4.5,1.8,0.4,3.1,2.9,35.4,Kobe Bryant,Regular Season
